# Exercise 1 - Modeling of human crowds

In [1]:
"""
Important notice: The code currently only works properly if a pedestrian is chosen BEFORE the target is chosen
"""
import arcade
import math

# Set the grid size
ROW_COUNT = 50
COLUMN_COUNT = 50

# This sets the WIDTH and HEIGHT of each grid location
WIDTH = 10
HEIGHT = 10

# This sets the margin between each cell
# and on the edges of the screen.
MARGIN = 2

#This specifies the screen size
SCREEN_HEIGHT = ROW_COUNT*HEIGHT+(ROW_COUNT+1)*MARGIN
SCREEN_WIDTH = COLUMN_COUNT*WIDTH+(COLUMN_COUNT+1)*MARGIN

neighbors = []
    


In [2]:
#This function prints the terms of the placed objects in the terminal

#For simplification and to sum up different terms that are in use
    #'0' refers to an empty cell
    #'1' refers to the color RED and a PEDESTRIAN
    #'2' refers to the color BLUE and an OBJECT
    #'3' refers to the color YELLOW and a TARGET
    
def terminology(index,row,column):
    if index == 1:
        print("There's a pedestrian in cell [{},{}]"
            .format(row,column))
    elif index == 2:
        print("There's an object in cell [{},{}]"
            .format(row,column))
    elif index == 3:
        print("The target is in cell [{},{}]"
            .format(row,column))
    else:
        print("Cell [{},{}] is clear"
             .format(row,column))

In [3]:
#This function is used to save all neighbors of a pedestrian
def save_neighbors(grid,neighbors,row,column):
    #Checks if the Object in a cell is a pedestrian
    if grid[row][column] == 1:
        neighbors[row][column].append([row+1,column+1])
        neighbors[row][column].append([row+1,column])
        neighbors[row][column].append([row+1,column-1])
        neighbors[row][column].append([row,column+1])
        neighbors[row][column].append([row,column-1])
        neighbors[row][column].append([row-1,column+1])
        neighbors[row][column].append([row-1,column])
        neighbors[row][column].append([row-1,column-1])
        
    return neighbors[row][column]


In [4]:
#Calculate distance of each neighbor of each pedestrian to the target
#Keep coordinates of neighbor with least distance and return in list
def calc_distance(neighbors,closest_neighbors,grid,row,column):
    #Pre-definition of variables used in this function
    distance_list = []
    least_distance = 100000
    list_index = 100
    
    for row_ in range(ROW_COUNT):
        for column_ in range(COLUMN_COUNT):
            
            #Checks if object in cell is a pedestrian
            if grid[row_][column_] == 1:
                
                #Iterates over all neighbors of the pedestrian and calculates their distance to the target
                for r in range(len(neighbors[row_][column_])):
                    distance = math.sqrt(((neighbors[row_][column_][r][0]-row)**2) + ((neighbors[row_][column_][r][1]-column)**2))
                    
                    #This if case is used to determine the closest neighbor
                    #After the for-loop the neighbors list-index is determined
                    #And its distance is saved as least_distance
                    if distance < least_distance:
                        least_distance = distance
                        list_index = r
                closest_neighbors[row_][column_].append(neighbors[row_][column_][list_index])
                    
    return closest_neighbors, least_distance

In [5]:
#This function updates the states of pedestrian to clear
#And the pedestrians closest neighbor to the target to pedestrian
#In short: This function moves the pedestrian one cell closer to the target
def update_objects(neighbors,grid,closest_neighbors):
    closest_neighbor = []
    row = -1
    column = -1
    #Iteration over all cells
    for row_ in range(ROW_COUNT):
        for column_ in range(COLUMN_COUNT):
            #Check if cell is pedestrian
            if grid[row_][column_] == 1:
                row = row_
                column = column_
                #Clear state
                neighbors[row_][column_].clear()
                grid[row][column] = 0
    #Update neighbor to pedestrian (move the pedestrian)            
    closest_row, closest_column = closest_neighbors[row][column][0]
    grid[closest_row][closest_column] = 1
                
    return grid,neighbors, closest_row, closest_column

In [6]:
class MyApplication(arcade.Window):

    #Initialization of the grid, neighbors, closest_neighbors as arrays
    #As well as variables used for computation
    def __init__(self, width, height):
        
        super().__init__(width, height)

        self.grid = []
        self.neighbors = []
        self.closest_neighbors = []
        self.distance = 1
        self.target_row = -1
        self.target_column = -1
        
        #Initialize 2D arrays for the grid, neighbors, closest_neighbors
        for row in range(ROW_COUNT):
            self.grid.append([])
            self.neighbors.append([])
            self.closest_neighbors.append([])
            for column in range(COLUMN_COUNT):
                self.grid[row].append(0)  # Append a cell
                self.neighbors[row].append([])
                self.closest_neighbors[row].append([])
                
        #Set background color for the grid
        arcade.set_background_color(arcade.color.BLACK)
        
        

    def on_draw(self):
        arcade.start_render()
        
        #Draw the grid
        for row in range(ROW_COUNT):
            for column in range(COLUMN_COUNT):
                #Figure out what color to draw the box according to
                #The colors / numbers specified in the beginning
                #Pedestrian / 1 / RED
                #Object / 2 / BLUE
                #Target / 3 / YELLOW
                #Clear / 0 / WHITE
                if self.grid[row][column] == 1:
                    color = arcade.color.RED
                elif self.grid[row][column] == 2:
                    color = arcade.color.BLUE
                elif self.grid[row][column] == 3:
                    color = arcade.color.YELLOW
                else:
                    color = arcade.color.WHITE

                #Calculate where to color the box
                x = (MARGIN + WIDTH) * column + MARGIN + WIDTH // 2
                y = (MARGIN + HEIGHT) * row + MARGIN + HEIGHT // 2

                # Draw the box
                arcade.draw_rectangle_filled(x, y, WIDTH, HEIGHT, color) 

                
    # When the user presses any key, this event fires:
    
    def on_key_press(self, symbol, modifiers):
    # "symbol" is the key being pressed by the user.
    #closest_row and closest_column specify the cell of the neighbor that is one step closer to the target
        closest_row = -1
        closest_column = -1
        if symbol == arcade.key.RIGHT:
            print("RIGHT")
            #If case is valid if pedestrian hasn't reached target yet
            if self.distance >= 1:
                self.grid,self.neighbors, closest_row, closest_column = update_objects(self.neighbors,self.grid,self.closest_neighbors)
                self.neighbors[closest_row][closest_column] = save_neighbors(self.grid,self.neighbors,closest_row,closest_column)
                self.closest_neighbors, self.distance = calc_distance(self.neighbors,self.closest_neighbors,self.grid,self.target_row,self.target_column)
        else:
            print("Press '->' to move one step towards the target.")
            
            
    # When the user clicks on a cell, this event fires:
    
    def on_mouse_press(self, x, y, button, modifiers):
        
        # Change the x/y screen(pixel) coordinates to grid coordinates
        column = x // (WIDTH + MARGIN)
        row = y // (HEIGHT + MARGIN)
        
        #
        print("Click coordinates: ({}, {}). Grid coordinates: ({}, {})"
              .format(x, y, row, column))

        if row < ROW_COUNT and column < COLUMN_COUNT:

            #Flip the object from nothing to pedestrian to object to target and back to object on each mouseclick
            if self.grid[row][column] == 0:
                self.grid[row][column] += 1
            elif self.grid[row][column] == 1:
                self.grid[row][column] += 1
            elif self.grid[row][column] == 2:
                #Save row and column of the target
                self.target_row = row
                self.target_column = column
                
                #Calculate dististance from neighbors to target and choose neighbor with smallest distance
                self.closest_neighbors, self.distance = calc_distance(self.neighbors,self.closest_neighbors,self.grid,row,column)
                self.grid[row][column]  += 1
            elif self.grid[row][column] == 3:
                self.grid[row][column] = 0
        #Save neighbors of a cell
        self.neighbors[row][column] = save_neighbors(self.grid,self.neighbors,row,column)


window = MyApplication(SCREEN_WIDTH, SCREEN_HEIGHT)

#keeps window open until user closes it
arcade.run()

Click coordinates: (136, 419). Grid coordinates: (34, 11)
Click coordinates: (322, 316). Grid coordinates: (26, 26)
Click coordinates: (322, 316). Grid coordinates: (26, 26)
Click coordinates: (322, 316). Grid coordinates: (26, 26)
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
